In [1]:
#import all package/library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE as fs
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
#import data and checking
raw_data = pd.read_csv("train.csv")
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          891 non-null    float64
 1   Cabin        204 non-null    object 
 2   Embarked     891 non-null    object 
 3   Fare         891 non-null    float64
 4   Name         891 non-null    object 
 5   Parch        891 non-null    int64  
 6   PassengerId  891 non-null    int64  
 7   Pclass       891 non-null    int64  
 8   Sex          891 non-null    object 
 9   SibSp        891 non-null    int64  
 10  Survived     891 non-null    float64
 11  Ticket       891 non-null    object 
 12  Title        891 non-null    object 
 13  Family_Size  891 non-null    int64  
dtypes: float64(3), int64(5), object(6)
memory usage: 97.6+ KB


In [3]:
#drop unique column
raw_data = raw_data.drop(columns=["Cabin","Name", "Ticket", "Title"])
raw_data

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Family_Size
0,22.0,S,7.2500,0,1,3,male,1,0.0,1
1,38.0,C,71.2833,0,2,1,female,1,1.0,1
2,26.0,S,7.9250,0,3,3,female,0,1.0,0
3,35.0,S,53.1000,0,4,1,female,1,1.0,1
4,35.0,S,8.0500,0,5,3,male,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
886,27.0,S,13.0000,0,887,2,male,0,0.0,0
887,19.0,S,30.0000,0,888,1,female,0,1.0,0
888,22.0,S,23.4500,2,889,3,female,1,0.0,3
889,26.0,C,30.0000,0,890,1,male,0,1.0,0


In [4]:
#change type data to numberic
raw_data["Sex"]= raw_data["Sex"].replace(["male", "female"], [0,1])
raw_data["Embarked"]= raw_data["Embarked"].replace(["S","C","Q"], [1,2,3])
raw_data


,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Family_Size
0,22.0,1,7.2500,0,1,3,0,1,0.0,1
1,38.0,2,71.2833,0,2,1,1,1,1.0,1
2,26.0,1,7.9250,0,3,3,1,0,1.0,0
3,35.0,1,53.1000,0,4,1,1,1,1.0,1
4,35.0,1,8.0500,0,5,3,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
886,27.0,1,13.0000,0,887,2,0,0,0.0,0
887,19.0,1,30.0000,0,888,1,1,0,1.0,0
888,22.0,1,23.4500,2,889,3,1,1,0.0,3
889,26.0,2,30.0000,0,890,1,0,0,1.0,0


In [5]:
#split data to variable X and Y, followed by ranking column data X
X = raw_data.loc[:,raw_data.columns !="Survived"]
Y = raw_data.iloc[:,8:9].values.ravel()
model_ls = LogisticRegression(max_iter=1000)
rfe = fs(model_ls)
rfe.fit(X,Y)
print(f'Support = {rfe.support_}')
print(f'Ranking = {rfe.ranking_}')

Support = [False  True False False False  True  True  True False]
Ranking = [4 1 5 3 6 1 1 1 2]


In [6]:
#Update data X to collect all ranking 1
X_new = raw_data[['Embarked', "Pclass","Sex","SibSp"]]
X_new

,Embarked,Pclass,Sex,SibSp
0,1,3,0,1
1,2,1,1,1
2,1,3,1,0
3,1,1,1,1
4,1,3,0,0
...,...,...,...,...
886,1,2,0,0
887,1,1,1,0
888,1,3,1,1
889,2,1,0,0


In [8]:
#make function to find best model Supervased Learning Classification
def supervased_classification(X,Y):
    pipeline_lr = Pipeline([("scaler",StandardScaler()),("PCA",PCA(n_components=2)),("lr",LogisticRegression(max_iter=1000))])
    pipeline_knn = Pipeline([("scaler",StandardScaler()),("PCA",PCA(n_components=2)),("knn",KNeighborsClassifier())])
    pipeline_nb = Pipeline([("scaler",StandardScaler()),("PCA",PCA(n_components=2)),("nb",GaussianNB())])
    pipeline_rf = Pipeline([("scaler",StandardScaler()),("PCA",PCA(n_components=2)),("rf",RandomForestClassifier())])
    pipeline_dt = Pipeline([("scaler",StandardScaler()),("PCA",PCA(n_components=2)),("dt",DecisionTreeClassifier())])
    pipelines = [pipeline_lr,pipeline_knn,pipeline_nb,pipeline_rf,pipeline_dt]
    best_score = 0
    dict_pipe = {0:"Logistic Linear", 1: "KNN", 2:"Naive Bayes", 3:"Random Forest", 4:"Decision Tree"}
    cv = StratifiedKFold(random_state=56, shuffle=True)
    score = cross_val_score(model_ls,X_new,Y,cv=cv,scoring="accuracy")
    for i,model in enumerate (pipelines):
        score = cross_val_score(model,X_new,Y,cv=cv,scoring="accuracy")
        score = np.mean(score)
        print(f"model {dict_pipe[i]} score AUC : {score}")
        if score > best_score:
            best_score = score
            best_pipeline = model
            best_machine = dict_pipe[i]
        print("-------")
    print(f"model terbaik adalah {best_machine} dengan score AUC {best_score}")
supervased_classification(X_new,Y)

model Logistic Linear score AUC : 0.7968300797187873
-------
model KNN score AUC : 0.784457975017262
-------
model Naive Bayes score AUC : 0.7968238026489234
-------
model Random Forest score AUC : 0.7912058251208337
-------
model Decision Tree score AUC : 0.7956939300734417
-------
model terbaik adalah Logistic Linear dengan score AUC 0.7968300797187873
